# ASL Project

In [1]:
#Import libraries
import numpy as np
import random

# Imports to view data
import cv2
import pandas as pd
from glob import glob

# Visualization
from keras.utils import print_summary
from matplotlib import pyplot as plt

#ML libraries
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
#Directory paths
TRAIN_DIR = "../Dataset/asl_alphabet_train/asl_alphabet_train"
TEST_DIR = "../Dataset/asl_alphabet_test"
MODEL_DIR = './Model'
MODEL_PATH = MODEL_DIR+"/Model1.h5"
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight1.h5"

In [3]:
# Set global variables
TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3) # add channel for RGB
CLASSES = 29
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64

## Data Augmentation

In [4]:
#Load Train dataset
train_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=VALIDATION_SPLIT
)

validation_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    validation_split=VALIDATION_SPLIT
)

train_generator = train_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, shuffle=True, subset="training")
val_generator = validation_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


## Model

In [5]:
#Define Model VGG16 Model
def VGG16_model_build():
    from keras.applications.vgg16 import VGG16
    from keras.layers import Input

    input_tensor = Input(shape=TARGET_DIMS)
    model = VGG16(input_tensor = input_tensor, weights=None, include_top=True, classes= CLASSES)
    return model

In [6]:
from keras.models import Model
base_model = VGG16_model_build()
x = base_model.output
#x = (Dense(CLASSES, activation='softmax'))(x)
model = Model(inputs=base_model.input, outputs=x)

#Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model.summary()
model.save(MODEL_PATH)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [8]:
#Checkpointer to save the best models
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*2 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*2 / BATCH_SIZE) )

model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=20, callbacks=[checkpointer], verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
39/39 [==============================] - 313s 8s/step - loss: 15.2746 - acc: 0.0329 - val_loss: 15.5137 - val_acc: 0.0375

Epoch 00001: val_loss improved from inf to 15.51367, saving model to ./Model/Model_Weight1.h5
Epoch 2/20
39/39 [==============================] - 320s 8s/step - loss: 15.6273 - acc: 0.0304 - val_loss: 15.6648 - val_acc: 0.0281

Epoch 00002: val_loss did not improve from 15.51367
Epoch 3/20
39/39 [==============================] - 325s 8s/step - loss: 15.5627 - acc: 0.0345 - val_loss: 15.5640 - val_acc: 0.0344

Epoch 00003: val_loss did not improve from 15.51367
Epoch 4/20
39/39 [==============================] - 318s 8s/step - loss: 15.6079 - acc: 0.0317 - val_loss: 15.4633 - val_acc: 0.0406

Epoch 00004: val_loss improved from 15.51367 to 15.46330, saving model to ./Model/Model_Weight1.h5
Epoch 5/20
39/39 [==============================] - 322s 8s/step - loss: 15.5369 - acc: 0.0361 - val_loss: 15.4633 - va

In [9]:
#Predict on validation dataset
predictions = model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J'] 64
15.614404678344727 , 0.03125


In [10]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [11]:
#Predict
test_generator.reset()
predictions = model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J'] 28


In [12]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(results)

                              Filename Predictions
0         asl_alphabet_test\A_test.jpg           J
1         asl_alphabet_test\B_test.jpg           J
2         asl_alphabet_test\C_test.jpg           J
3         asl_alphabet_test\D_test.jpg           J
4         asl_alphabet_test\E_test.jpg           J
5         asl_alphabet_test\F_test.jpg           J
6         asl_alphabet_test\G_test.jpg           J
7         asl_alphabet_test\H_test.jpg           J
8         asl_alphabet_test\I_test.jpg           J
9         asl_alphabet_test\J_test.jpg           J
10        asl_alphabet_test\K_test.jpg           J
11        asl_alphabet_test\L_test.jpg           J
12        asl_alphabet_test\M_test.jpg           J
13        asl_alphabet_test\N_test.jpg           J
14        asl_alphabet_test\O_test.jpg           J
15        asl_alphabet_test\P_test.jpg           J
16        asl_alphabet_test\Q_test.jpg           J
17        asl_alphabet_test\R_test.jpg           J
18        asl_alphabet_test\S_t

In [13]:
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 3.571428571428571


In [14]:
#Define Model VGG16 Model with pretrained weight
def VGG16_model_build_weight():
    from keras.applications.vgg16 import VGG16
    from keras.layers import Input

    input_tensor = Input(shape=TARGET_DIMS)
    model = VGG16(input_tensor = input_tensor, weights='imagenet', include_top=False)
    return model

from keras.models import Model
weight_base_model = VGG16_model_build_weight()
x1 = weight_base_model.output
#Add the fully-connected layers 
x1 = Flatten(name='flatten')(x1)
x1 = Dense(4096, activation='relu', name='fc1')(x1)
x1 = Dense(4096, activation='relu', name='fc2')(x1)
x1 = Dense(CLASSES, activation='softmax', name='predictions')(x1)
weight_model = Model(inputs=weight_base_model.input, outputs=x1)

#Compile the model
weight_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

weight_model.summary()
MODEL_PATH = MODEL_DIR+"/Model1-withweight.h5"
weight_model.save(MODEL_PATH)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [15]:
#Checkpointer to save the best models
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight1-withweight.h5"
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*2 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*2 / BATCH_SIZE) )

weight_model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=10, callbacks=[checkpointer], verbose=1)

Epoch 1/10
39/39 [==============================] - 327s 8s/step - loss: 15.2754 - acc: 0.0337 - val_loss: 15.3122 - val_acc: 0.0500

Epoch 00001: val_loss improved from inf to 15.31219, saving model to ./Model/Model_Weight1-withweight.h5
Epoch 2/10
39/39 [==============================] - 321s 8s/step - loss: 15.6144 - acc: 0.0312 - val_loss: 15.3122 - val_acc: 0.0500

Epoch 00002: val_loss did not improve from 15.31219
Epoch 3/10
39/39 [==============================] - 332s 9s/step - loss: 15.5757 - acc: 0.0337 - val_loss: 15.3626 - val_acc: 0.0469

Epoch 00003: val_loss did not improve from 15.31219
Epoch 4/10
39/39 [==============================] - 317s 8s/step - loss: 15.5434 - acc: 0.0357 - val_loss: 15.5060 - val_acc: 0.0380

Epoch 00004: val_loss did not improve from 15.31219
Epoch 5/10
39/39 [==============================] - 317s 8s/step - loss: 15.5821 - acc: 0.0333 - val_loss: 15.4129 - val_acc: 0.0437

Epoch 00005: val_loss did not improve from 15.31219
Epoch 6/10
39/39 

In [16]:
#Predict on validation dataset
predictions = weight_model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = weight_model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'] 64
15.866250038146973 , 0.015625


In [17]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [18]:
#Predict
test_generator.reset()
predictions = weight_model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'] 28


In [19]:
filenames=test_generator.filenames
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 3.571428571428571
